## Original Data- this data can be obtained from the Blockchain after appropriate parsing.


blockhash.txt -- enumeration of all blocks in the blockchain, 277443 rows, 4 columns: 


 	blockID -- id used in this database (0 -- 277442, continous) 
 	bhash -- block hash (identifier in the blockchain, 64 hex characters) 
 	btime -- creation time (from the blockchain) 
 	txs -- number of transactions 

txhash.txt -- transaction ID and hash pairs, 30048983 rows, 2 columns: 


 	txID -- id used in this database (0 -- 30048982, continous) 
 	txhash -- transaction hash used in the blockchain (64 hex characters) 

addresses.txt -- BitCoin address IDs, 24618959 rows, 2 columns: 


 	addrID -- id used in this database (0 -- 24618958, continous, the address with addrID == 0 is invalid /blank, not used/) 
 	addr -- string representation of the address (alphanumeric, maximum 35 characters; note that the IDs are NOT ordered by the addr in any way) 


***
tx.txt -- enumaration of all transactions, 30048983 rows, 4 columns: 


 	txID -- transaction ID (from the txhash.txt file) 
 	blockID -- block ID (from the blockhash.txt file) 
 	n_inputs -- number of inputs 
 	n_outputs -- number of outputs 

***
txin.txt -- list of all transaction inputs (sums sent by the users), 65714232 rows, 3 columns: 


 	txID -- transaction ID (from the txhash.txt file) 
 	addrID -- sending address (from the addresses.txt file) 
 	value -- sum in Satoshis (1e-8 BTC -- note that the value can be over 2^32, use 64-bit integers when parsing) 

***
txout.txt -- list of all transaction outputs (sums received by the users), 73738345 rows, 3 columns: 


 	txID -- transaction ID (from the txhash.txt file) 
 	addrID -- receiving address (from the addresses.txt file) 
 	value -- sum in Satoshis (1e-8 BTC -- note that the value can be over 2^32, use 64-bit integers when parsing) 

***
txtime.txt -- transaction timestamps (obtained from the blockchain.info site), 30048983 rows, 2 columns: 


 	txID -- transaction ID (from the txhash.txt file) 
 	unixtime -- unix timestamp (seconds since 1970-01-01) 
_______


## Computed data - this data is obtained from the above data

contraction.txt -- list of addresses possibly belonging to the same user, 24618959 rows, 2 columns: 


 	addrID -- address ID (from the addresses.txt file) 
 	userID -- ID of identified user (not continuos, each two addrID which belong to the same "user" appear as inputs in the same transaction at least once) 

balances.txt -- balances of nodes after 277,443 blocks (on 2013.12.28.), 24617959 rows, 2 columns: 


 	addrID -- address ID (from the addresses.txt file) 
 	balance -- balance in Satoshis (1e-8 BTC -- note that the value can be over 2^32, use 64-bit integers when parsing) 

degree.txt -- node degrees (number of distinct transaction partners), 24575385, 3 columns: 


 	addrID -- address ID (from the addresses.txt file) 
 	indeg -- indegree (number of distinct addresses which appear as inputs in transactions where this address appears as output) 
 	outdeg -- outdegree (number of distinct addresses which appear as outputs in transactions where this address appears as input) 

txedge.txt -- edges constructed from the transactions: a transaction with 2 inputs and 3 outputs results in 6 edges (all possible combinations), an edge may appear multiple times, with the corresponding transaction IDs, 129178908 rows, 3 columns: 


 	txID -- transaction ID in which this edge appears 
 	addrin -- sending address 
 	addrout -- receiving address 
    blockID -- Block ID

txedgeunique.txt -- edges constructed from the transactions; each edge appears only once, 89220163 rows, 2 columns:


 	addrin -- sending address 
 	addrout -- receiving address 

# Analyzing the Blockchain
This study looks at the blockchain as it was at the end of year 2013, wherever possible, extra information has been provided

In [50]:
import time
import mmap
import pymongo
from pymongo import MongoClient


client = MongoClient()
client = MongoClient('localhost', 27017)

"""
this is our database which has following collections/tables/databases. For further description, 
match the names with the original dataset

'degree',
'addresses',
'addressCount',
'txout',
'txtime',
'blockhash',
'contraction',
'txedge',
'txin',
'txhash',
'tx'
"""

db = client.blockchain

"""
The database below is based on info we scraped from the blockchain.info webste. The number of total 
tags are 3557. Of which 3316 tags have been verified.
"""
db_meta = client.info

In [51]:
"""
We index the table based on entries we query. This makes the lookup almost as fast

For comparison, a lookup of a wallet address among 24.5 million addresses takes....
...0.001255 seconds, which is almost equal to in-memory python dict fetch. Yeah!!!
"""
#db.tx.create_index( "txID" )
#db.addresses.create_index( "addrID")
#db.txout.create_index("txID")
#db.txin.create_index("txID")
#db.txedge.create_index("txID")
#db.txedge.create_index("addrin")
#db.txtime.create_index("txID")
#db.txhash.create_index("txID")
#db.tx.create_index("txID")
#db.degree.create_index("addrID")
#db.contraction.create_index("userID")
#db.contraction.create_index("addrID")

## How many addresses can we tag?
In this notebook, we start with 3557 addresses, then we go on to tag clusters. We are interested in finding how many addresses (don't forget 24.6 mill) can we tag with just 3557 publically tagged addresses. 
Tagging is a crucial step, for instance, if we know one address belongs to a bad guy, then we find bunch of addresses belonging to that guy, and we tag them all with the individuals owenership. We'll later develop a very simple strategy to give risk rating to different addresses.

In [117]:
#Find the clusters with largest sizes.
documents = db.contraction.find()
addressCount = {}
for i in range(documents.count()):
    userID = documents.next()['userID']
    addressCount[userID] = addressCount.get(userID, 0) + 1

#There is no need to sort as shown below
sorted_addressCount = sorted(addressCount.items(), key=operator.itemgetter(1))

In [ ]:
#Create a database of clusters.
posts = []
for each, another in sorted_addressCount:
    posts.append({'userID': each, 'addressCount': another})
    
#Bad idea: over use of memory
db.addressCount.insert_many(posts)

#db.addressCount.find().count()
#12137803- total clusters

In [ ]:
#Meta info of addresses
#I had to do this to migrate the old database into the new one
#You might want to see how our meta-info database is set up with 
#future in mind.
posts = []
while cursor:
    post = cursor.next()
    posts.append({
        'address': post['address'][0],
        'meta_info': [{'info_source': 'blockchain_info_public',
                       'tag': post['tag'][0],
                       'ref_link': post['url_link'][0],
                       'verified': post['blockchain_info_verified']}]
    })

### How many tags actually?
Because we have tags 3557 tags collected this week, and our dataset is only until the end of 2013, we pick only the relevant tags - 1541 (shown below).  

In [123]:
t = time.time()

cursor = db_meta.meta.find()
known_addresses = []

for entry in cursor:
    if db.addresses.find_one({'addr': entry['address']}):
        known_addresses.append(entry['address'])
        
print("{0} addresses tagged.".format(len(known_addresses)))
print("It took {0} seconds to carry out the above operations".format(time.time()-t))

1541 addresses tagged.
It took 1.8386666774749756 ms to carry out the above operations


In [124]:
# We obtained a list of bitcoin addresses and their tags above.
# Now we map those actual bitcoin addresses to our internal id representation.

t = time.time()

known_addresses_id = [db.addresses.find_one({'addr': x})['addrID'] for x in known_addresses]

print("It took {0} seconds to carry out the above operations".format(time.time()-t))

It took 0.7090697288513184 seconds to carry out the above operations


In [127]:
#Let's connected our internal ids[tagged above] to our clusters

t = time.time()

known_addresses_cluster_id = [(x, db.contraction.find_one({'addrID': x})['userID']) for x in known_addresses_id]

print("It took {0} seconds to carry out the above operation".format(time.time()-t))

#This is not necessary, I did it for my own analysis.
known_addresses_cluster_id.sort(key=lambda x: x[1])

It took 0.7048068046569824 seconds to carry out the above operation


In [95]:
# Now the fun part.

"""
We do a single round reverse lookup. That is, we pick an address, find its matching cluster.
Because our original hypothesis was that our clustering was done based on the
owner of the addresses. Also, because we know the tag for one (or more) addresses belonging
to a cluster, we can tag all the addresses belonging to that cluster. 
Let's see what we get. 
"""

In [157]:
# Let's find a representative address of a cluster i.e. one address per cluster.
cluster_with_one_address = [(cluster, addrID) for addrID, cluster in known_addresses_cluster_id]
seen = []
cluster_representative_addrID = []
for each in cluster_with_one_address:
    if each[0] not in seen:
        cluster_representative_addrID.append((each[0],each[1]))
        seen.append(each[0])
        
len(cluster_representative_addrID)                                            

792

In [167]:
total_tagged = []
for each in cluster_representative_addrID:
    for each in db.contraction.find({'userID': each[0]}):
        total_tagged.append((each['addrID'], each['userID']))
len(total_tagged)       

1625753

### We tagged 1625753 individual address with just 792 relevant tags!!!

In [176]:
#Let's do some verification.
db.addresses.find_one({'addrID': known_addresses_cluster_id[100][1]})

{'_id': ObjectId('5aaa60272e007d12bfa8c167'),
 'addr': '112QNTtSrKgqua5s7uCDHasJKqXsaHg93R',
 'addrID': 3414}